In [ ]:
# -*- coding: utf-8 -*-
# -*- author : Vincent Roduit -*-
# -*- date : 2023-11-25 -*-
# -*- Last revision: 2023-11-25 -*-
# -*- python version : 3.11.6 -*-
# -*- Description: Try notebook-*-

# <center> CS -433 Machine Learning </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Road Segmentation </center>
--- 

### Preparing environment for Google Colaboratory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/ml-project-2-team-slo/source

### Imports

In [ ]:
#import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import matplotlib.pyplot as plt

import torch.nn.functional as F
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
from preprocessing_helper import *

%load_ext autoreload
%autoreload 2

In [ ]:
#import files
from load_datas import*
from constants import*
from visualization import*
from data_processing import*
from data_augmentation import*
from training_utils import*

## 1. Load and preprocess datas

In [ ]:
imgs, gt_imgs = load_datas(NB_IMAGES_MAX)

In [ ]:
imgs_train = imgs[:80]
gt_imgs_train = gt_imgs[:80]
imgs_test = imgs[80:100]
gt_imgs_test = gt_imgs[80:100]

In [ ]:
imgs_train.shape
imgs_test.shape

In [ ]:
visualize(imgs, gt_imgs, index=6)

In [ ]:
X, y = image_generator(imgs_train, gt_imgs_train, window_size=100,nb_batches=1, batch_size=32, upsample=True)

In [ ]:
x_val, y_val = image_generator(imgs_test, gt_imgs_test, 16,nb_batches=50, batch_size=32, upsample=True)

In [ ]:
visualize_patch(X[2].transpose(1,2,0))

In [ ]:
train_dataloader = DataLoader(
    dataset=list(zip(X, y)),
    batch_size=32,
    shuffle=False,
    num_workers=2)
validate_dataloader = DataLoader(
    dataset=list(zip(x_val, y_val)),
    batch_size=32,
    shuffle=False,
    num_workers=2)

In [ ]:
class CorrectBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    WINDOW_SIZE = 100

    def __init__(self, block, num_blocks, num_classes=2):
        super().__init__()
        self.in_planes = 64
        shape = (self.WINDOW_SIZE, self.WINDOW_SIZE, 3)
        # Adjust the input channels to match the input size (3 channels)
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=64,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        # Use AdaptiveAvgPool2d to dynamically adapt to the output size
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.linear = nn.Linear(512, num_classes)  # Adjust the input size to 512

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avg_pool(out)
        out = out.view(out.size(0), -1).contiguous()
        out = self.linear(out)
        return out

In [ ]:
# initialize the model
model = ResNet(block=CorrectBlock, num_blocks=[2, 2, 2, 2], num_classes=2)

# define the optimizer
optimizer = torch.optim.RAdam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

train_losses, valid_losses, valid_accs = fit(
    model=model,
    train_dataloader=train_dataloader,
    valid_dataloader=validate_dataloader,
    optimizer=optimizer,
    epochs=20,
    device=DEVICE,
    scheduler = scheduler
)

In [ ]:
sub_imgs = load_test_datas()
sub_imgs_processed = ProcessingData(sub_imgs)
sub_imgs_processed.create_patches()
sub_imgs_processed.permute_axis()

In [ ]:
subs_loader = DataLoader(
    dataset=sub_imgs_processed.imgs_patches,
    batch_size=32,
    shuffle=False,
    num_workers=2)

In [ ]:
sub_preds = predict_test_set(model=model, test_dataloader=subs_loader, device=DEVICE)

In [ ]:
sub_preds = sub_preds.ravel()

In [ ]:
from post_processing import*
nb_imgs = 50
images_filenames = save_pred_as_png(sub_preds,nb_imgs,PATCH_SIZE,label_to_img)

In [ ]:
masks_to_submission('submission.csv', *images_filenames)